<a href="https://colab.research.google.com/github/rooshanriaz/CS-351L-AI-Lab-GitHub-Repository_2022506/blob/main/Rooshan_Riaz_CS351L_Lab_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Introduction to Search in AI**

**Name: Rooshan Riaz**

**Reg No: 2022506**

## **Lab Task: Implementation of A* Algorithm on a Maze Solver Game**

classic Maze Solver game, where the player or AI uses the A algorithm* to find the shortest path from a start position (S) to the exit (E) in a randomly generated maze.

# **How It Works:**

**Generate a Maze:** A random maze is created with walls and paths, and the start and exit positions are set.

**Solve the Maze:** The A* algorithm is used to find the shortest path from start to exit, exploring cells and marking them as visited.

**Print Progress:** The maze is printed at each step to show the current state, including visited cells.

**Display Results:** The final maze is printed with the path from start to exit marked, if a path is found.

# **Explanation:**

**1. Maze Generation:**
generate_maze(rows, cols): This function creates a maze of the specified dimensions (rows x cols). It populates the grid with walls and paths randomly. The starting position (S) is placed at the top-left corner, and the exit (E) is placed near the bottom-right corner.

**2. Printing the Maze:**
print_maze(maze): This function prints the maze in a visually clear format with borders. It helps in visualizing the maze layout, including walls, paths, and symbols representing the start, exit, and visited cells.

**3. A* Algorithm:**
a_star_solve(maze, start_pos, exit_pos): This function implements the A* search algorithm to find the shortest path from the start position to the exit. It uses a priority queue (min-heap) to explore nodes based on their total cost (f), which is the sum of the cost from the start (g) and the heuristic estimate to the exit (h). The algorithm:
Expands nodes from the priority queue.
Marks cells as visited during exploration.
Continues until the exit is reached or all possible paths are explored.
Returns the path found or None if no path exists.

**4. Heuristic Function:**
heuristic(pos1, pos2): This function calculates the Manhattan distance between two positions, which estimates the cost from a given node to the exit. It is used to prioritize nodes in the A* algorithm.

**5. Path Reconstruction:**
reconstruct_path(node): Once the exit is reached, this function traces back from the exit node to the start node, constructing the path taken.

**6. Main Execution:**
run_maze_solver(): This function orchestrates the maze generation, solving, and printing. It:
Generates a random maze.
Solves the maze using the A* algorithm.
Marks and displays the path found (if any) in the final maze.



In [3]:
import heapq
import random
import time

# Define the maze dimensions
ROWS = 8
COLS = 9

# Symbols for the maze
WALL = '█'
PATH = '.'
START = 'S'
EXIT = 'E'
VISITED = '*'
FINAL_PATH = 'P'

# Directions for movement (right, down, left, up)
DIRECTIONS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0  # Cost from start to current node
        self.h = 0  # Heuristic cost to the end
        self.f = 0  # Total cost (f = g + h)

    def __lt__(self, other):
        return self.f < other.f

def generate_maze(rows, cols):
    maze = [[WALL if random.random() < 0.2 else PATH for _ in range(cols)] for _ in range(rows)]

    # Set start and exit positions
    start_pos = (0, 0)
    exit_pos = (rows - 1, cols - 2)
    maze[start_pos[0]][start_pos[1]] = START
    maze[exit_pos[0]][exit_pos[1]] = EXIT
    return maze, start_pos, exit_pos

def print_maze(maze):
    # Print the maze with borders and improved spacing
    print("-" * (COLS * 4 + 1))
    for row in maze:
        row_str = "| " + " | ".join(row) + " |"
        print(row_str)
        print("-" * (COLS * 4 + 1))

def heuristic(pos1, pos2):
    # Manhattan distance heuristic
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def is_valid_position(maze, pos):
    # Check if the position is within the maze and is a walkable path or exit
    return 0 <= pos[0] < len(maze) and 0 <= pos[1] < len(maze[0]) and maze[pos[0]][pos[1]] in [PATH, EXIT]

def a_star_solve(maze, start_pos, exit_pos):
    # Priority queue (min-heap)
    open_set = []
    heapq.heappush(open_set, (0, Node(start_pos)))

    # Set of visited positions
    closed_set = set()

    iterations = 0
    while open_set:
        iterations += 1
        # Pop the node with the lowest f score
        _, current_node = heapq.heappop(open_set)
        current_pos = current_node.position

        # Mark as visited
        if maze[current_pos[0]][current_pos[1]] == PATH:
            maze[current_pos[0]][current_pos[1]] = VISITED

        print(f"Step {iterations}:")
        print_maze(maze)
        time.sleep(0.3)  # Slow down to visualize step-by-step

        # If we reached the exit, reconstruct the path
        if current_pos == exit_pos:
            return reconstruct_path(current_node)

        closed_set.add(current_pos)

        # Explore neighbors
        for direction in DIRECTIONS:
            neighbor_pos = (current_pos[0] + direction[0], current_pos[1] + direction[1])

            if neighbor_pos in closed_set or not is_valid_position(maze, neighbor_pos):
                continue

            # Create a new node for the neighbor
            neighbor_node = Node(neighbor_pos, current_node)
            neighbor_node.g = current_node.g + 1  # Cost from start to neighbor
            neighbor_node.h = heuristic(neighbor_pos, exit_pos)  # Heuristic to exit
            neighbor_node.f = neighbor_node.g + neighbor_node.h

            # Add to the open set
            heapq.heappush(open_set, (neighbor_node.f, neighbor_node))

    return None  # No path found

def reconstruct_path(node):
    path = []
    while node:
        path.append(node.position)
        node = node.parent
    return path[::-1]  # Return the path from start to exit

# Main function to run the maze solver
def run_maze_solver():
    # Generate a random maze
    maze, start_pos, exit_pos = generate_maze(ROWS, COLS)
    print("Initial Maze:")
    print_maze(maze)

    # Solve the maze using A* algorithm
    path = a_star_solve(maze, start_pos, exit_pos)

    # If a path is found, mark the path on the maze
    if path:
        print("Path found!")
        for pos in path:
            if maze[pos[0]][pos[1]] == VISITED:
                maze[pos[0]][pos[1]] = FINAL_PATH  # Mark the path with 'P'
        maze[start_pos[0]][start_pos[1]] = START  # Ensure start position is marked
        maze[exit_pos[0]][exit_pos[1]] = EXIT    # Ensure exit position is marked
    else:
        print("No path found.")

    # Print the final maze with the path
    print("Final Maze with Path:")
    print_maze(maze)

# Run the maze solver
run_maze_solver()





Initial Maze:
-------------------------------------
| S | █ | . | . | . | . | █ | . | . |
-------------------------------------
| . | . | . | . | . | . | . | . | . |
-------------------------------------
| . | . | . | . | . | . | . | . | . |
-------------------------------------
| . | █ | █ | . | . | . | █ | . | . |
-------------------------------------
| . | . | . | . | . | . | . | █ | . |
-------------------------------------
| . | █ | . | . | . | . | █ | . | . |
-------------------------------------
| . | . | . | . | . | █ | █ | . | . |
-------------------------------------
| . | . | . | . | █ | █ | . | E | . |
-------------------------------------
Step 1:
-------------------------------------
| S | █ | . | . | . | . | █ | . | . |
-------------------------------------
| . | . | . | . | . | . | . | . | . |
-------------------------------------
| . | . | . | . | . | . | . | . | . |
-------------------------------------
| . | █ | █ | . | . | . | █ | . | . |
----------------------------

# **Explanation of Output**

**Walls (█):** Solid blocks that cannot be traversed.

**Path (.):** Represents the open, walkable areas of the maze.

**Visited cells (*)**: As the algorithm explores the maze, cells are marked with *.

**Final Path (P):** The optimal path from S to E, marked with P.